In [ ]:
#!cp drive/MyDrive/data_clustered.csv data_clustered.csv

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_clustered = pd.read_csv('data_clustered.csv') # Файл из выхода Kmeans.ipynb

In [ ]:
data_clustered

,Unnamed: 0,Unnamed: 0.1,LotArea,GarageArea,MSZoning,LotFrontage,Condition1,HouseStyle,SalePrice,YearBuilt,YrSold,Neighborhood,Label_5,Label_9
0,0,479,5925,672,RM,50.0,Norm,1Story,89471,1937,2007,OldTown,1,5
1,1,793,9158,474,RL,76.0,Norm,1Story,225000,2007,2007,Somerst,3,0
2,2,339,12400,297,RL,66.0,Feedr,1Story,155000,1958,2009,NAmes,2,3
3,3,142,8520,720,RL,71.0,Artery,1.5Fin,166000,1952,2010,NAmes,2,3
4,4,1453,17217,0,RL,90.0,Norm,1Story,84500,2006,2006,Mitchel,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,831,3180,480,FV,30.0,Norm,2Story,151000,2005,2006,Somerst,2,3
1456,1456,522,5000,420,RM,50.0,Feedr,1.5Fin,159000,1947,2006,BrkSide,2,3
1457,1457,1217,8640,552,FV,72.0,Norm,1Story,229456,2009,2009,Somerst,3,0
1458,1458,162,12182,532,RL,95.0,Norm,1Story,220000,2005,2010,NridgHt,3,6


Возьмем Label_5 как лэйбл кластеризации на 5 кластеров из прошлой задачи

In [ ]:
labels = data_clustered['Label_5']

Как фичу возьму SalePrice

In [ ]:
feature = data_clustered['SalePrice']

## Оценка среднего фичи по датасету с помощью Бутстрапа

In [ ]:
feature = data_clustered['SalePrice']

Возьму и 2000 раз насэмплю элементов из датасета
Каждый раз посчитаю среднее и сохраню это в список means

In [ ]:
n_bootstrap = 2000
n_sample = feature.shape[0]

means = np.zeros((n_bootstrap, ))

for i in range(n_bootstrap):
    feats = np.random.choice(feature, n_sample, replace=True)
    means[i] = np.mean(feats)
    

### Pivotal

Для Pivotal версии считаем, что means теперь распределено нормально, и посчитаем доверительный интервал через выборочное среднее и стандартное отклонение массива means

In [ ]:
mean_means = np.mean(means)
std_mean = np.std(means)

left_pl_bound = mean_means - 1.96 * std_mean
right_pl_bound = mean_means + 1.96 * std_mean

Получается такой доверительный интервал

In [ ]:
print(f'SalePrice is in boundary [{left_pl_bound}; {right_pl_bound}] with 95% confidence')

SalePrice is in boundary [176718.65206984174; 185050.59132604866] with 95% confidence


### Non-pivotal

Для этой версии, отсортирую means

In [ ]:
sorted_means = np.sort(means)

In [ ]:
sorted_means

array([173858.65684932, 174191.02465753, 174308.30821918, ...,
       187042.76232877, 187115.22808219, 187175.43767123])

Возьму 2.5% и 97.5% перцентили полученных средних и они и будут задавать 95%-доверительный интервал

In [ ]:
lower_bound_index = int(sorted_means.shape[0] * 0.025)
upper_bound_index = int(sorted_means.shape[0] * 0.975)

left_np_bound = sorted_means[lower_bound_index]
right_np_bound = sorted_means[upper_bound_index]

Получается следующий результат:

In [ ]:
print(f'SalePrice is in boundary [{left_np_bound}; {right_np_bound}] with 95% confidence')

SalePrice is in boundary [176776.77465753426; 185291.49589041097] with 95% confidence


### **Результаты**

Результаты для pivotal и non-pivotal версий получаются схожи, скорее всего в среднее оба они попали достаточно хорошо


## Сравнение средних меж кластерами

Возьму первый и второй кластер и их средние буду сравнивать

In [ ]:
feature_label_1 = feature[labels == 1]
feature_label_2 = feature[labels == 2]

Для каждого из двух кластеров в течение 2000 итераций буду считать среднее по сэмплированным данным из этих лейблов

Для первого кластера

In [ ]:
n_bootstrap = 2000
n_sampled = feature_label_1.shape[0]

means_1 = np.zeros((n_bootstrap, ))

for i in range(n_bootstrap):
    feats = np.random.choice(feature_label_1, n_sampled, replace=True)
    means_1[i] = np.mean(feats)



Для второго кластера

In [ ]:
n_bootstrap = 2000
n_sampled = feature_label_2.shape[0]

means_2 = np.zeros((n_bootstrap, ))

for i in range(n_bootstrap):
    feats = np.random.choice(feature_label_2, n_sampled, replace=True)
    means_2[i] = np.mean(feats)

Посчитаю разницу между соответствующими массивами средних. Это я делаю, потому что я буду проверять, попадает ли 0 в доверительный интервал разниц запуска для первого кластера и для второго

In [ ]:
diffs = means_1 - means_2

### Pivotal

Как и в прошлый раз для Pivotal считаем, что среднее становится похоже на гауссиану и высчитываем доверительный интервал исходя из этого

In [ ]:
mean_diffs = np.mean(diffs)
std_diffs = np.std(diffs)

left_pl_bound = mean_diffs - 1.96 * std_diffs
right_pl_bound = mean_diffs + 1.96 * std_diffs

Результаты

In [ ]:
print(f'Diff between Saleprices of first and second clusters is in boundary [{left_pl_bound}; {right_pl_bound}] with 95% confidence')

Diff between Saleprices of first and second clusters is in boundary [-59142.491642781424; -54377.96970936607] with 95% confidence


### Non-pivotal

Опять же отсортируем диффы

In [ ]:
sorted_diffs = np.sort(diffs)

Возьмем 2.5% и 97.5% перцентили, и они и задают нам доверительный интервал

In [ ]:
lower_bound_index = int(sorted_diffs.shape[0] * 0.025)
upper_bound_index = int(sorted_diffs.shape[0] * 0.975)

left_np_bound = sorted_diffs[lower_bound_index]
right_np_bound = sorted_diffs[upper_bound_index]

Результаты

In [ ]:
print(f'Diff between Saleprices of first and second clusters is in boundary [{left_np_bound}; {right_np_bound}] with 95% confidence')

Diff between Saleprices of first and second clusters is in boundary [-59034.63390724971; -54340.81206616864] with 95% confidence


### **Результаты**

В обоих случаях 95%-доверительный интервал находится очень далеко от нуля. Таким образом, я могу отвергнуть гипотезу, что в этих двух кластерах среднее значение SalePrice совпадает

## Сравниваю среднее в кластере с глобальным средним с помощью Бутстрапа

Возьму ту же самую фичу и буду сравнивать ее среднее в первом кластере с глобальным средним

In [ ]:
feature_label_1 = feature[labels == 1]

Для первого кластера и всего датасета в течение 2000 итераций буду считать среднее по сэмплированным данным из соответствующего подмножества данных

Для всего множества

In [ ]:
n_bootstrap = 2000
n_sampled = feature.shape[0]

means_global = np.zeros((n_bootstrap, ))

for i in range(n_bootstrap):
    feats = np.random.choice(feature, n_sampled, replace=True)
    means_global[i] = np.mean(feats)

Для первого кластера

In [ ]:
n_bootstrap = 2000
n_sampled = feature_label_1.shape[0]

means_1 = np.zeros((n_bootstrap, ))

for i in range(n_bootstrap):
    feats = np.random.choice(feature_label_1, n_sampled, replace=True)
    means_1[i] = np.mean(feats)

Посчитаю разницу между соответствующими списками средних для того, чтоб потом проверить, попадает ли 0 в доверительный интервал

In [ ]:
diffs = means_global - means_1

### Pivotal

Опять же для pivotal версии посчитаю среднее диффов, и посчитаю как для гауссианы доверительный интервал

In [ ]:
mean_diffs = np.mean(diffs)
std_diffs = np.std(diffs)

left_pl_bound = mean_diffs - 1.96 * std_diffs
right_pl_bound = mean_diffs + 1.96 * std_diffs

Результаты

In [ ]:
print(f'Diff between Saleprices of first cluster and full dataset is in boundary [{left_pl_bound}; {right_pl_bound}] with 95% confidence')

Diff between Saleprices of first cluster and full dataset is in boundary [58744.240761002206; 67661.07228763503] with 95% confidence


### Non-pivotal

Для non-pivotal версии вновь отсортирую диффы

In [ ]:
sorted_diffs = np.sort(diffs)

Возьму 2.5% и 97.5% перцентили диффов, они будут давать доверительный интервал

In [ ]:
lower_bound_index = int(sorted_diffs.shape[0] * 0.025)
upper_bound_index = int(sorted_diffs.shape[0] * 0.975)

left_np_bound = sorted_diffs[lower_bound_index]
right_np_bound = sorted_diffs[upper_bound_index]

Результаты

In [ ]:
print(f'Diff between Saleprices of first cluster and full dataset is in boundary [{left_np_bound}; {right_np_bound}] with 95% confidence')

Diff between Saleprices of first cluster and full dataset is in boundary [58771.65921221026; 67648.63341944353] with 95% confidence


### Результаты

Оба доверительных интервала дают право отвергнуть гипотезу, что фактор SalePrice имеет одинаковое среднее в первом кластере и во всем датасете

## Выводы

Оба способа подсчета доверительных интервалов с помощью бутстрапа дают возможность сравнивать два разных датасета на похожесть, и довольно точно вычислять среднее